# SGA CRISPR Cas Systems

In [ ]:
import glob
import os
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from Bio import SearchIO
from Bio import SeqIO
import json
from collections import defaultdict
import skbio
import subprocess as sp
from collections import OrderedDict
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Seq import Seq

In [ ]:
rootdir = "to_fill_in"

starting folders in rootdir:
1. All_genomes
2. All_genomes_proteins
3. metadata
4. Scripts (with parallel-fastq-dump, process_reads_bbmap, crispr v6)

In [ ]:
def makedir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

In [ ]:
#Creating dictionary to match scaffolds to a bin name
scaf2bin_dic = {}
for file in glob.glob(rootdir + "All_genomes/*fna"):
    name = file.replace(rootdir + "All_genomes/", "").replace(".fna", "")
    for record in SeqIO.parse(open(file), "fasta"):
        scaf2bin_dic[record.description.split(" ")[0]] = name

In [ ]:
#Creating dictionary to match SGA taxonomy with bin name

bin2tax_dict = {}
bin_2_tax_df = pd.read_csv(rootdir + "metadata/bin2tax.tsv", "\t", names=["bin", "tax"])
for key, row in bin_2_tax_df.iterrows():
    bin2tax_dict[row["bin"]] = row["tax"]

In [ ]:
makedir(rootdir + "Scripts")

## Preparing the SGA Database

### Dereplication of the database

In [ ]:
#alex to fill out?

### Predicting protiens in each genome

In [ ]:
# alex too fill out?

### Genome Annotation

In [ ]:
# KEGG, UniRef100, UniProt

In [ ]:
#Gene taxonomic predictions USEARCH with UniRef100

### Tree Construction

## CRISPR Cas Systems in SGA

### CRISPR Cas Finder (CCF)

In [ ]:
#cat all genomes
print("cat {0}All_genomes/* > {0}All_genomes.fna".format(rootdir))

In [ ]:
#cat all proteins
print("cat {0}All_genomes_proteins/* > {0}ALL_genome_proteins.faa".format(rootdir))

In [ ]:
makedir(rootdir + "CCF_output_all_genomes")

manually curated scaffolds with CCF 3 and 4 arrays to ensure they originated from SGA bacteria

In [ ]:
#read in results
ccf_output_df = pd.read_csv(rootdir + "/CCF_output_all_genomes/TSV/Crisprs_REPORT.tsv", sep="\t")
#scaf 2 bin
ccf_output_df["Bin"] = ccf_output_df.Sequence.map(scaf2bin_dic)
#bin 2 tax
ccf_output_df["Lineage"] = ccf_output_df.Bin.map(bin2tax_dict)
#subset columns
ccf_output_df = ccf_output_df[["Sequence","Bin","Lineage", "CRISPR_Start", "CRISPR_End", "Spacers_Nb", "Mean_size_Spacers", "Evidence_Level"]].rename(columns={"Sequence":"Scaffold"})

#adding in which scaffolds passed QC
CCF_scaffold_QC_final_list_df = pd.read_csv(rootdir + "/metadata/CCF_scaffold_QC_final_list.csv")
CCF_scaffold_QC_final_list_df = CCF_scaffold_QC_final_list_df.dropna()
CCF_scaffold_QC_final_list_df = CCF_scaffold_QC_final_list_df[CCF_scaffold_QC_final_list_df["include"]]

QC_dict = {}
for key, row in CCF_scaffold_QC_final_list_df.iterrows():
    QC_dict[row["ccf_scaffold_name"]] = row["include"]

ccf_output_df["Passed_manual_curation"] = ccf_output_df.Scaffold.map(QC_dict)
ccf_output_df["Passed_manual_curation"] = ccf_output_df["Passed_manual_curation"].fillna(False)

ccf_output_df = ccf_output_df.sort_values(by=["Passed_manual_curation", "Lineage", "Evidence_Level"], ascending=False)

ccf_output_df.to_csv(rootdir + "CCF_output_manual_curation.csv", index=False)

### Searching for Cas genes

In [ ]:
#hmm parse function
def parse_hmm(result_table):
    temp = {}
    count = 0
    # parse each result file using searchio
    for result in SearchIO.parse(result_table, "hmmer3-text"):
        for item in result.hits:
            temp[count] = {"gene": item.id, "score": item.bitscore, "eval": item.evalue, "query": result.id, "des": result.description, "accession": result.accession}
            count += 1
    return(pd.DataFrame.from_dict(temp, orient="index"))

In [ ]:
#parse the tigrfam hmm results
tigrfam_hmm_df = parse_hmm(rootdir + "AlltigrfamHMMprofiles_to_cpr_db.txt")

#filtering tigrfam results based on evalue and dropping duplicates
tigrfam_hmm_df_duplicates_dropped = tigrfam_hmm_df.sort_values(by=["eval"]).drop_duplicates(subset=["gene"])

#only cas protein list
tigrfam_hmm_df_duplicates_dropped_cas_only = tigrfam_hmm_df_duplicates_dropped[tigrfam_hmm_df_duplicates_dropped["des"].str.contains("CRISPR")]

#simplify description
tigrfam_hmm_df_duplicates_dropped_cas_only["des_simp"] = tigrfam_hmm_df_duplicates_dropped["des"].apply(lambda x: x.split(":")[0])

In [ ]:
#manually constructed dictionary to categorize the called cas genes
des_simp2_cas_category_dict = {"cas2":"cas2","cas_Csn1":"cas9","cas1_NMENI":"cas1","cas_Cpf1":"cpf1","cas1_PREFRAN":"cas1","cas4_PREFRAN":"cas4","cas_Csn2":"csn2","cas1":"cas1","cas_cas6":"cas6","cas7_TM1809":"cas7","cas_TM1810_Csm2":"csm2","cas_TM1811_Csm1":"cas10","cas_TM1807_csm5":"cas7","cas5_csm4":"cas5","cas4":"cas4","cas1_HMARI":"cas1","TIGR03986":"TIGR03986","cas_csf4":"csf4","cas3_core":"cas3","cas_Csd1":"cas8c","cas_CXXC_CXXC":"cst1","cas3_HD":"cas3","cas_RAMP_Cmr4":"cas7","cas_TM1791_cmr6":"cas7","cas_cmr3":"cas5","cas_Cas5d":"cas5","cas_TM1794_Cmr2":"cas10","cas_CT1132":"ct1132","cas5_csf3":"cas5","cas_Csh2":"cas7","TIGR03984":"TIGR03984","cas_TM1812":"TM1812","cas_csx3":"csx3","cas_Csy4":"cas6","cas_Csy2":"csy2","casB_cse2":"casB_cse2","cas1_DVULG":"cas1","cas3_yersinia":"cas3","cas8u_csf1":"csf1","cas_csm6":"csm6","cas3_GSU0051":"cas3","cas_Csy3":"csy3","cas_csx17":"csx17","cas_Csy1":"csy1","cas_MJ0381":"MJ0381","casA_cse1":"casA_cse1","cas7_csf2":"csf2","cas1_YPEST":"cas1","cas_Csd2":"cas7","cas_Cas5h":"cas5"}

In [ ]:
#subset table
sub_tigrfam_hmm_df_duplicates_dropped_cas_only = tigrfam_hmm_df_duplicates_dropped_cas_only[["gene","des_simp"]]
#grab scaffold name
sub_tigrfam_hmm_df_duplicates_dropped_cas_only["scaffold"] = sub_tigrfam_hmm_df_duplicates_dropped_cas_only["gene"].apply(lambda x: x.rsplit("_", 1)[0])

In [ ]:
#adding in bin and standarizing the Cas gene calls
sub_tigrfam_hmm_df_duplicates_dropped_cas_only = sub_tigrfam_hmm_df_duplicates_dropped_cas_only.reset_index(drop=True)
sub_tigrfam_hmm_df_duplicates_dropped_cas_only["bin"] = sub_tigrfam_hmm_df_duplicates_dropped_cas_only["scaffold"].map(scaf2bin_dic)

sub_tigrfam_hmm_df_duplicates_dropped_cas_only["cas_type"] = ""
for inda in sub_tigrfam_hmm_df_duplicates_dropped_cas_only.index:
    sub_tigrfam_hmm_df_duplicates_dropped_cas_only["cas_type"][inda] = des_simp2_cas_category_dict[sub_tigrfam_hmm_df_duplicates_dropped_cas_only["des_simp"][inda]]

In [ ]:
#subsetting to bins from our CCF QC scaffolds 
CCF_output_df = pd.read_csv(rootdir + "/metadata/CCF_scaffold_QC_final_list.csv")
CCF_QC_bins = CCF_output_df[CCF_output_df.include == True]["bin"].unique()

sub_tigrfam_hmm_df_duplicates_dropped_cas_only = sub_tigrfam_hmm_df_duplicates_dropped_cas_only[sub_tigrfam_hmm_df_duplicates_dropped_cas_only["bin"].isin(CCF_QC_bins)]

In [ ]:
#manually curated all scaffolds containing Cas genes to ensure they were from SGA bacteria
#read in these results
manual_curation_parsed_df = pd.read_csv(rootdir + "metadata/complete_cas_systems_manual_curation_parsed.csv")
manually_curated_scaffs = manual_curation_parsed_df[manual_curation_parsed_df.include == True]["scaffold"].unique()

#subsetting to manually curated scaffolds
sub_tigrfam_hmm_df_duplicates_dropped_cas_only = sub_tigrfam_hmm_df_duplicates_dropped_cas_only[sub_tigrfam_hmm_df_duplicates_dropped_cas_only.scaffold.isin(manually_curated_scaffs)]

### Complete CRISPR-Cas system profiling

In [ ]:
#getting cas_type counts
sub_tigrfam_hmm_df_duplicates_dropped_cas_only_pivot = sub_tigrfam_hmm_df_duplicates_dropped_cas_only.groupby(["bin", "cas_type"], as_index=False).aggregate({"gene":"count"})
sub_tigrfam_hmm_df_duplicates_dropped_cas_only_pivot.columns = ["bin", "cas_type", "count"]
sub_tigrfam_hmm_df_duplicates_dropped_cas_only_pivot = sub_tigrfam_hmm_df_duplicates_dropped_cas_only_pivot.pivot("bin", "cas_type", "count").fillna(0).reset_index()

In [ ]:
#calculating which systems and how many are in each genome
bin2systems_number = {}
bin2systems = {}
sub_tigrfam_hmm_df_duplicates_dropped_cas_only_pivot_calc = sub_tigrfam_hmm_df_duplicates_dropped_cas_only_pivot.set_index("bin")
for key, row in sub_tigrfam_hmm_df_duplicates_dropped_cas_only_pivot_calc.iterrows():
    bin_sys_list = []
    #type IIA
    while row["cas9"] >0 and row["cas1"] > 0 and row["cas2"] > 0 and row["csn2"] > 0:
        bin_sys_list.append("Type II-A")
        row["cas9"] = row["cas9"] - 1
        row["cas1"] = row["cas1"] - 1
        row["cas2"] = row["cas2"] - 1
        row["csn2"] = row["csn2"] - 1
        continue
    #type II-C1
    while row["cas9"] >0 and row["cas1"] > 0 and row["cas2"] > 0 and row["csn2"] == 0:
        bin_sys_list.append("Type II-C1")
        row["cas9"] = row["cas9"] - 1
        row["cas1"] = row["cas1"] - 1
        row["cas2"] = row["cas2"] - 1
        continue
    #type V-A
    while row["cpf1"] >0 and row["cas1"] > 0 and row["cas2"] > 0 and row["cas4"] > 0:
        bin_sys_list.append("Type V-A")
        row["cpf1"] = row["cpf1"] - 1
        row["cas1"] = row["cas1"] - 1
        row["cas2"] = row["cas2"] - 1
        row["cas4"] = row["cas4"] - 1
        continue
    #type III-A
    while row["cas10"] >0 and row["cas7"] > 0 and row["cas5"] > 0 and row["csm2"] > 0:
        bin_sys_list.append("Type III-A")
        row["cas10"] = row["cas10"] - 1
        row["cas7"] = row["cas7"] - 1
        row["cas5"] = row["cas5"] - 1
        row["csm2"] = row["csm2"] - 1
        continue
    #type III-B
    while row["cas10"] >0 and row["cas7"] > 0 and row["cas5"] > 0 and row["csm2"] == 0:
        bin_sys_list.append("Type III-B")
        row["cas10"] = row["cas10"] - 1
        row["cas7"] = row["cas7"] - 1
        row["cas5"] = row["cas5"] - 1
        continue
    bin2systems[key] = bin_sys_list
    bin2systems_number[key] = len(bin_sys_list)

In [ ]:
#make systems dataframe
complete_cas_systems_df = sub_tigrfam_hmm_df_duplicates_dropped_cas_only_pivot[["bin"]].reset_index(drop=True)
complete_cas_systems_df["Complete_CRISPR_Cas_Systems"] = complete_cas_systems_df.bin.map(bin2systems)
complete_cas_systems_df["Number_of_Complete_CRISPR_Cas_Systems"] = complete_cas_systems_df.bin.map(bin2systems_number)


#which bins have complete systems?
complete_cas_systems_df["complete_cas_systems_boolean"] = ""
for indA in complete_cas_systems_df.index:
    if str(complete_cas_systems_df["Complete_CRISPR_Cas_Systems"][indA]) == "[]":
        complete_cas_systems_df["complete_cas_systems_boolean"][indA] = False
    else:
        complete_cas_systems_df["complete_cas_systems_boolean"][indA] = True


#add tax
complete_cas_systems_df["tax"] = complete_cas_systems_df.bin.map(bin2tax_dict)

In [ ]:
#adding in genes
bin2genes_dict = {}
for bin in sub_tigrfam_hmm_df_duplicates_dropped_cas_only.bin.unique():
    sub_df = sub_tigrfam_hmm_df_duplicates_dropped_cas_only[sub_tigrfam_hmm_df_duplicates_dropped_cas_only.bin == bin]
    sub_df = sub_df.sort_values(by="gene", ascending=True)
    genes_dict = {}
    for key, row in sub_df.iterrows():
        genes_dict[row["gene"]] = row["des_simp"]
    bin2genes_dict[bin] = genes_dict

complete_cas_systems_df["cas_genes"] = complete_cas_systems_df.bin.map(bin2genes_dict)

In [ ]:
#add in the CCF info
ccf_output_df = pd.read_csv(rootdir + "CCF_output_manual_curation.csv")
#manually curated scaffolds
ccf_output_df = ccf_output_df[ccf_output_df.Passed_manual_curation == True]

#make nested CCF dict
ccf_dict = {}
for bin in ccf_output_df.Bin.unique():
    ccf_dict[bin] = {}
    sub_df = ccf_output_df[ccf_output_df.Bin == bin]
    array_count = 0
    for key, row in sub_df.iterrows():
        array_count += 1
        ccf_dict[bin]["CRISPR_array_" + str(array_count)] = {}
        ccf_dict[bin]["CRISPR_array_" + str(array_count)]["Scaffold"] = row["Scaffold"]
        ccf_dict[bin]["CRISPR_array_" + str(array_count)]["CRISPR_Start"] = row["CRISPR_Start"]
        ccf_dict[bin]["CRISPR_array_" + str(array_count)]["CRISPR_End"] = row["CRISPR_End"]
        ccf_dict[bin]["CRISPR_array_" + str(array_count)]["Evidence_Level"] = row["Evidence_Level"]
        ccf_dict[bin]["CRISPR_array_" + str(array_count)]["Spacers_Nb"] = row["Spacers_Nb"]
        ccf_dict[bin]["CRISPR_array_" + str(array_count)]["Mean_size_Spacers"] = row["Mean_size_Spacers"]

complete_cas_systems_df["CRISPR_Arrays"] = complete_cas_systems_df.bin.map(ccf_dict)

In [ ]:
#adding in if the genomes are redundant or not
drep_clusters = pd.read_csv(rootdir + "metadata/drep_df.csv")
drep_clusters = drep_clusters.drop_duplicates(subset=["secondary_cluster"])
drep_genomes = drep_clusters.genome.str.replace(".fna", "")

complete_cas_systems_df["Nonredundant"] = ""
for indA in complete_cas_systems_df.index:
    if complete_cas_systems_df["bin"][indA] in list(drep_genomes):
        complete_cas_systems_df["Nonredundant"][indA] = True
    else:
        complete_cas_systems_df["Nonredundant"][indA] = False

In [ ]:
#save the dataframe
complete_cas_systems_df.to_csv(rootdir + "Complete_Crispr_cas_systems_with_subtypes.csv", index=False)

### Clinker visualization of the CRISPR-Cas Systems

In [ ]:
#create a scaffold2seq dict
scaffold2seq = {}
for file in glob.glob(rootdir + "All_genomes/*fna"):
    for record in SeqIO.parse(open(file), "fasta"):
        scaffold2seq[record.description.split(" ")[0]] = str(record._seq)

In [ ]:
#tigrfam dict
gene2tigr = {}
for key, row in tigrfam_hmm_df_duplicates_dropped.iterrows():
    gene2tigr[row["gene"]] = row["des"]

In [ ]:
#cat all proteins in the database
print("cat {0}/All_genome_proteins/* > {0}All_genome_proteins.faa".format(rootdir))

In [ ]:
#parsing the faa files
genes_dict = defaultdict(list)
for record in SeqIO.parse(rootdir + "ALL_genome_proteins.faa", "fasta"):
    genes_dict["gene"].append(record.id)
    genes_dict["gene_number"].append(int(record.id.rsplit("_", 1)[1]))
    genes_dict["scaffold"].append(record.id.rsplit("_", 1)[0])
    genes_dict["bin"].append(scaf2bin_dic[record.id.rsplit("_", 1)[0]])
    genes_dict["start"].append(record.description.split(" # ")[1])
    genes_dict["stop"].append(record.description.split(" # ")[2])
    genes_dict["direction"].append(record.description.split(" # ")[3])
    genes_dict["seq"].append(str(record._seq))

genes_df = pd.DataFrame(genes_dict)

#grab tigrfam annotations and cas descriptions
genes_df["des"] = genes_df["gene"].map(gene2tigr)
genes_df["des_simp"] = genes_df["des"].apply(lambda x: str(x).split(":")[0])
genes_df["cas"] = genes_df["des_simp"].map(des_simp2_cas_category_dict)

#seeing if bin is redundant
genes_df["Nonredundant"] = genes_df["bin"].apply(lambda x: True if x in set(drep_genomes) else False)

In [ ]:
#looking for complete systems in the manually curated scaffolds, sliding window of 5 genes, grabbing all genes within the cas genes too 
systems = {}
for scaffold in manually_curated_scaffs:
    scaf_df = genes_df[genes_df.scaffold == scaffold]

    #subset to only cas genes
    cas_df = scaf_df[~scaf_df.cas.isna()].reset_index(drop=True)
    system_count = 1
    systems[scaffold] = {}
    cas_gene_list = []
    for indA in cas_df.index:
        cas_gene_list.append(int(cas_df["gene_number"][indA]))

        #check if you are at the end of the cas_df
        if indA == (len(cas_df) -1):
            systems[scaffold][system_count] = list(range(min(cas_gene_list), max(cas_gene_list) +1))
            continue

        #checking if the next cas gene number is within 5 of the current cas gene, if not add 1 to system count, reset_system_gene_list
        if int(cas_df["gene_number"][indA + 1]) >= int(cas_df["gene_number"][indA]) + 5:
            systems[scaffold][system_count] = list(range(min(cas_gene_list), max(cas_gene_list) +1))
            system_count += 1
            cas_gene_list = []

In [ ]:
#map system_number back to genes_df
gene_to_system = {}
for scaffold in systems.keys():
    for system_number in systems[scaffold].keys():
        for gene_number in systems[scaffold][system_number]:
            gene_to_system[scaffold + "_" + str(gene_number)] = scaffold + "_system_" + str(system_number)

genes_df["crispr_cas_system"] = genes_df.gene.map(gene_to_system)

In [ ]:
#profiling the systems
scafs = []
for scaf in manually_curated_scaffs:
    sub_df = genes_df[genes_df.scaffold == scaf]
    #type II-A
    if all(x in set(sub_df["cas"]) for x in ["cas9", "cas1", "cas2", "csn2"]):
        scafs.append(scaf)
        continue

    #type II-C1:
    elif all(x in set(sub_df["cas"]) for x in ["cas9", "cas1", "cas2"]):
        scafs.append(scaf)
        continue

    #type V-A
    elif all(x in set(sub_df["cas"]) for x in ["cpf1", "cas1", "cas2", "cas4"]):
        scafs.append(scaf)

In [ ]:
#make folders
makedir(rootdir + "clinker_crispr_cas/")
makedir(rootdir + "clinker_crispr_cas/type_two_A")
makedir(rootdir + "clinker_crispr_cas/type_two_C1")
makedir(rootdir + "clinker_crispr_cas/type_five_A")

In [ ]:
#make genebank files
def make_genebank_record(dataframe, scaffold, crispr_cas_system):
    record = SeqRecord(Seq(scaffold2seq[scaffold]), id=crispr_cas_system, name=crispr_cas_system, description=crispr_cas_system, annotations={"molecule_type": "DNA"})
    for key, row in dataframe.iterrows():
        gene_dict = OrderedDict()
        gene_dict['product'] = row["des_simp"]
        gene_dict['codon_start'] = 1
        gene_dict['gene'] = row["gene"]
        gene_dict['translation'] = row["seq"]
        gene = SeqFeature(FeatureLocation(start=int(row["start"]), end=int(row["stop"])), type='CDS', strand=int(row["direction"]), qualifiers=gene_dict)
        record.features.append(gene)
    return record

gene_bank_scafs = []
for crispr_cas_system in genes_df.crispr_cas_system.unique():
    sub_df = genes_df[genes_df.crispr_cas_system == crispr_cas_system][genes_df.Nonredundant == True]
    scaffold = str(crispr_cas_system).split("_system_")[0]
    #type II-A systems:
    if all(x in set(sub_df["cas"]) for x in ["cas9", "cas1", "cas2", "csn2"]):
        record = make_genebank_record(sub_df, scaffold, crispr_cas_system)
        with open(rootdir + "clinker_crispr_cas/type_two_A/{0}.gb".format(crispr_cas_system), "w") as file:
            SeqIO.write(record, file, 'genbank')
        gene_bank_scafs.append(scaffold)
        continue

    #type II-C1:
    elif all(x in set(sub_df["cas"]) for x in ["cas9", "cas1", "cas2"]):
        record = make_genebank_record(sub_df, scaffold, crispr_cas_system)
        with open(rootdir + "clinker_crispr_cas/type_two_C1/{0}.gb".format(crispr_cas_system), "w") as file:
            SeqIO.write(record, file, 'genbank')
        gene_bank_scafs.append(scaffold)
        continue

    #type V-A
    elif all(x in set(sub_df["cas"]) for x in ["cpf1", "cas1", "cas2", "cas4"]):
        record = make_genebank_record(sub_df, scaffold, crispr_cas_system)
        with open(rootdir + "clinker_crispr_cas/type_five_A/{0}.gb".format(crispr_cas_system), "w") as file:
            SeqIO.write(record, file, 'genbank')
        gene_bank_scafs.append(scaffold)

### BLAST our curated cas genes to NCBI nr database

In [ ]:
makedir(rootdir + "blast_cas_genes_to_NCBI")

In [ ]:
#make a fasta of the curated cas genes

with open(rootdir + "blast_cas_genes_to_NCBI/cas_genes.faa", "w") as file:
    for record in SeqIO.parse(rootdir + "ALL_genome_proteins.faa", "fasta"):
        if record.id in list(sub_tigrfam_hmm_df_duplicates_dropped_cas_only.gene):
            file.write(">{0}\n{1}\n".format(record.id, str(record._seq)))

In [ ]:
#read the results
blast_output_df = skbio.io.read(rootdir + 'blast_cas_genes_to_NCBI/cas_genes.blast', format="blast+6", into=pd.DataFrame, default_columns=True)
#sort by pident, drop duplicates
blast_output_df = blast_output_df.sort_values(by=["qseqid", "pident"], ascending = [True, False]).drop_duplicates(subset="qseqid")

#at least 75% coverage
blast_output_df["coverage"] = blast_output_df["length"]/(abs(blast_output_df["send"] - blast_output_df["sstart"]))
blast_output_df = blast_output_df[blast_output_df.coverage >= 0.75]

#add in cas type
gene2cas_dict = {}
for key, row in sub_tigrfam_hmm_df_duplicates_dropped_cas_only.iterrows():
    gene2cas_dict[row["gene"]] = row["cas_type"]
blast_output_df["cas_type"] = blast_output_df.qseqid.map(gene2cas_dict)
#add in scaffold and bin and taxonomy
blast_output_df["scaffold"] = blast_output_df.qseqid.apply(lambda x: x.rsplit("_", 1)[0])
blast_output_df["bin"] = blast_output_df.scaffold.map(scaf2bin_dic)
blast_output_df["tax"] = blast_output_df.bin.map(bin2tax_dict)


#dropping redundant genomes
blast_output_df = blast_output_df.reset_index(drop=True)
blast_output_df["Nonredundant"] = ""
for indA in blast_output_df.index:
    if blast_output_df["bin"][indA] in list(drep_genomes):
        blast_output_df["Nonredundant"][indA] = True
    else:
        blast_output_df["Nonredundant"][indA] = False

blast_output_df = blast_output_df[blast_output_df.Nonredundant == True]

In [ ]:
#plot cas type to pident

palette = {"Gracilibacteria":"tab:green",
           "Saccharibacteria":"tab:blue", 
           "Absconditabacteria":"tab:purple"}

sns.set_theme(style="whitegrid")
plt.figure(figsize=(12,5))
sns.stripplot(x="cas_type", y="pident",hue="tax", data=blast_output_df, jitter=0.3, linewidth=1, palette=palette, size =7, order=["cas1", "cas2","cas9","csn2", "cpf1", "cas4", "cas6", "cas10", "cas7", "cas5", "csm2"])
#sns.boxplot(x="cas_type", y="pident", data=blast_output_df, order=["cas1", "cas2","cas9","csn2", "cpf1", "cas4", "cas6", "cas10", "cas7", "cas5", "csm2"])
sns.despine()
plt.xlabel('')
plt.ylabel('Percent Identity (%)')

## Extracting spacers + Spacer array expansion steps

CCF Spacers

In [ ]:
makedir(rootdir + "CCF_output_all_genomes/spacer_fastas")

In [ ]:
#extract spacers from CCF evidence level 3 and 4 
with open(rootdir + "/CCF_output_all_genomes/result.json", 'r') as f:
    abs12_json = json.load(f)
for abs12 in abs12_json['Sequences']:
    for abs12_a in abs12['Crisprs']:
        if abs12_a["Evidence_Level"] == 4 or abs12_a["Evidence_Level"] == 3:
            for abs12_b in abs12_a['Regions']:
                if "Spacer" in abs12_b["Type"]:
                    fasta = open(rootdir + 'CCF_output_all_genomes/spacer_fastas/' + abs12['Version'] + "_" + str(abs12_b["Start"]) + '_CCF_spacer.fna', "w")
                    header = (">" + abs12['Version'] + "_" + str(abs12_b["Start"]))
                    sequence = (abs12_b['Sequence'])
                    fasta.write(header + "\n" + sequence + "\n")
                    fasta.close()

In [ ]:
#getting ccf spacers from qc scaffolds 
#extracting spacers via pullseq
script = open(rootdir + '/Scripts/extracting_ccf_spacers_from_qc_scaffolds.sh', "w")
for x in CCF_scaffold_QC_final_list_df["ccf_scaffold_name"]:
        pullseq = "pullseq -i {0}/CCF_output_all_genomes/all_ccf_spacers_3_4.fna -g ".format(rootdir)
        pullseq2 = " >> {0}/CCF_output_all_genomes/all_ccf_spacers_3_4_qc_scaffolds_only.fna".format(rootdir)
        script.write(pullseq + x + pullseq2 + "\n")
script.close()

run the above script

### Spacer array expansion

In [ ]:
makedir(rootdir + "spacer_expansion")

In [ ]:
#read in metadata info
reassemble_info_df = pd.read_csv(rootdir + "metadata/reassemble_genomes_from_reads_info.csv")
#remove unecessary columns
reassemble_info_df = reassemble_info_df[["bin", "run_id", "read_path", "assembly_path"]]
#fillna
reassemble_info_df = reassemble_info_df.fillna("None")

In [ ]:
makedir(rootdir + "spacer_expansion/raw_reads/")

In [ ]:
#Downloading reads from some publically avalible genomes
script = open(rootdir + 'Scripts/download_reads_parallel_fastq_dump_split_files.sh', "w")
for indA in reassemble_info_df.index:
    bin_name = reassemble_info_df["bin"][indA]
    run_id = reassemble_info_df["run_id"][indA]
    if "SRR" in reassemble_info_df["run_id"][indA]:
        if reassemble_info_df["read_path"][indA] != "None":
            parallel_fastq_dump = "{0}Scripts/parallel-fastq-dump --split-files -s <run_id> -t 16 -O {0}/spacer_expansion/raw_reads/ --gzip".format(rootdir)
            parallel_fastq_dump = parallel_fastq_dump.replace("<run_id>", run_id)
            script.write("#" + bin_name + "_" + run_id + "\n" + parallel_fastq_dump + "\n")
script.close()

run download_reads_parallel_fastq_dump_split_files.sh

In [ ]:
#quality control reads #split-files
script = open(rootdir + 'Scripts/quality_control_reads_split_files.sh', "w")
for indA in reassemble_info_df.index:
    bin_name = reassemble_info_df["bin"][indA]
    run_id = reassemble_info_df["run_id"][indA]

    if "SRR" in reassemble_info_df["run_id"][indA]:
        if reassemble_info_df["read_path"][indA] != "None":
            quality_control_1 = "{0}Scripts/process_reads_bbmap.rb -p 16 -z --create-fa {0}spacer_expansion/raw_reads/<run_id>_1_quality_controlled.fasta --basename {0}/spacer_expansion/raw_reads/<run_id>_1".format(rootdir)
            quality_control_1 = quality_control_1.replace("<run_id>", run_id)
            quality_control_2 = "{0}Scripts/process_reads_bbmap.rb -p 16 -z --create-fa /{0}spacer_expansion/raw_reads/<run_id>_2_quality_controlled.fasta --basename {0}spacer_expansion/raw_reads/<run_id>_2".format(rootdir)
            quality_control_2 = quality_control_2.replace("<run_id>", run_id)
            script.write("#" + bin_name + "_" + run_id + "\n" + quality_control_1 + "\n" + quality_control_2 + "\n")
script.close()

run quality_control_reads_split_files.sh

In [ ]:
makedir(rootdir + "spacer_expansion/assembly/")

In [ ]:
#assembly of fastq dump genomes #split_reads
script = open(rootdir + 'Scripts/assembly_of_fastq_dump_genomes_split_reads.sh', "w")
for indA in reassemble_info_df.index:
    bin_name = reassemble_info_df["bin"][indA]
    run_id = reassemble_info_df["run_id"][indA]

    if "SRR" in reassemble_info_df["run_id"][indA]:
        if reassemble_info_df["read_path"][indA] != "None":
            mkdir = "mkdir {0}/spacer_expansion/assembly/<run_id>/".format(rootdir)
            mkdir = mkdir.replace("<run_id>", run_id)
            megahit = "sbatch --wrap 'megahit -t 48 -1 {0}spacer_expansion/raw_reads/<run_id>_trim_clean.PE.1.fastq.gz -2 {0}spacer_expansion/raw_reads/<run_id>_trim_clean.PE.2.fastq.gz -o {0}/spacer_expansion/assembly/<run_id>/megahit/ --out-prefix <run_id>'".format(rootdir)
            megahit = megahit.replace("<run_id>", run_id)
            script.write("#" + bin_name + "_" + run_id + "\n" + mkdir + "\n" + megahit + "\n")
script.close()

run assembly_of_fastq_dump_genomes_split_reads.sh

In [ ]:
#mapping with bowtie 2 fastq #split reads
script = open(rootdir + 'Scripts/mapping_of_fastq_dump_genomes_split_reads.sh', "w")
for indA in reassemble_info_df.index:
    bin_name = reassemble_info_df["bin"][indA]
    run_id = reassemble_info_df["run_id"][indA]
    if "SRR" in reassemble_info_df["run_id"][indA]:
        if reassemble_info_df["read_path"][indA] != "None":
            mkdir_2 = "mkdir {0}/spacer_expansion/assembly/<run_id>/bt2/".format(rootdir)
            mkdir_2 = mkdir_2.replace("<run_id>", run_id)
            bt2_build = "bowtie2-build {0}/spacer_expansion/assembly/<run_id>/megahit/<run_id>.contigs.fa {0}/spacer_expansion/assembly/<run_id>/bt2/<run_id>.contigs.fa".format(rootdir)
            bt2_build = bt2_build.replace("<bin_name>", bin_name).replace("<run_id>", run_id)
            bt2 = "sbatch --wrap '/shared/software/bin/bowtie2 -p 48 -x {0}/spacer_expansion/assembly/<run_id>/bt2/<run_id>.contigs.fa -1 {0}spacer_expansion/raw_reads/<run_id>_trim_clean.PE.1.fastq.gz -2 {0}spacer_expansion/raw_reads/<run_id>_trim_clean.PE.2.fastq.gz 2> {0}/spacer_expansion/assembly/<run_id>/bt2/<run_id>_log_file.log | shrinksam | samtools view -S -b > {0}/spacer_expansion/assembly/<run_id>/bt2/<run_id>.bam'".format(rootdir)
            bt2 = bt2.replace("<bin_name>", bin_name).replace("<run_id>", run_id)
            script.write("#" + bin_name + "_" + run_id + "\n" + mkdir_2 + "\n" + bt2_build + "\n" + bt2 + "\n")
script.close()

run mapping_of_fastq_dump_genomes_split_reads.sh

In [ ]:
makedir(rootdir + "spacer_expansion/expansion_output/")

In [ ]:
#running lin Xing's script V6 on fastq_dump_genomes
script = open(rootdir + "Scripts/lin_xing_v6_script_of_fastq_dump_genomes.sh", "w")
for indA in reassemble_info_df.index:
    bin_name = reassemble_info_df["bin"][indA]
    run_id = reassemble_info_df["run_id"][indA]
    if "SRR" in reassemble_info_df["run_id"][indA]:
        if reassemble_info_df["read_path"][indA] != "None":
            assembly_path = "{0}/spacer_expansion/assembly/<run_id>/megahit/<run_id>.contigs.fa".replace("<run_id>", run_id).format(rootdir)
            bam_path = "{0}/spacer_expansion/assembly/<run_id>/bt2/<run_id>.bam".replace("<run_id>", run_id).format(rootdir)
            lin_xing_script = "{0}Scripts/CRISPR.v6.py -f <assembly_path> -m <bam_path> -c 16 -n 16 -o {0}/spacer_expansion/expansion_output/<bin_name>/".format(rootdir)
            lin_xing_script = lin_xing_script.replace("<assembly_path>", assembly_path).replace("<bam_path>", bam_path).replace("<bin_name>", bin_name)
            script.write("#" + bin_name + "_" + run_id + "\n" + lin_xing_script + "\n")
script.close()

run lin_xing_v6_script_of_fastq_dump_genomes.sh

### Blast assembled scaffolds to publically avalible scaffolds

In [ ]:
makedir(rootdir + "spacer_expansion/blast_output/")

In [ ]:
#blast script
file = open(rootdir + "Scripts/blast_assembled_scaffolds_to_public_scaffolds.sh", "w")
for indA in reassemble_info_df.index:

    if "SRR" in reassemble_info_df["run_id"][indA]:
        if reassemble_info_df["read_path"][indA] != "None":

            bin_name = reassemble_info_df["bin"][indA]
            run_id = reassemble_info_df["run_id"][indA]
            public_genome_path = "{0}/All_genomes/<bin_name>.fna".replace("<bin_name>", bin_name).format(rootdir)
            db_path = "{0}/spacer_expansion/assembly/<run_id>/megahit/<run_id>.contigs.fa".replace("<run_id>", run_id).format(rootdir)
            blast_db = "makeblastdb -in <db_path> -dbtype nucl"
            blast_db = blast_db.replace("<db_path>", db_path)
            blast_command = "blastn -task 'blastn-short' -query <public_genome_path> -db <db_path> -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qlen slen' -out /groups/banfield/projects/multienv/cpr/2020/tm7_sr1_gracili/Env_CPR_Alex/db_v4/Blast_assembled_scaffolds_public_output/blast_output/<bin_name>_<run_id>_ALL_results.csv -evalue 0.003 -word_size 7 -gapopen 10 -gapextend 2 -penalty -1 -max_target_seqs 1000 -num_threads 16".replace("<bin_name>", bin_name).replace("<run_id>", run_id)
            blast_command = blast_command.replace("<public_genome_path>", public_genome_path).replace("<db_path>", db_path)
            file.write("#" + bin_name + "\n" + blast_db + "\n" + blast_command + "\n")
file.close()

run blast_assembled_scaffolds_to_public_scaffolds.sh

In [ ]:
#read in blast results and concatonate
blast_results_dfs = []
for file in glob.glob(rootdir + "spacer_expansion/blast_output/*"):
    blast_df = skbio.io.read(file, format="blast+6", into=pd.DataFrame, default_columns=False,  columns=['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore', 'qlen', 'slen'])
    blast_results_dfs.append(blast_df)

all_blast_results_df = pd.concat(blast_results_dfs)

#caluculate coverage
all_blast_results_df["Coverage"] = all_blast_results_df['length']/all_blast_results_df['slen']
all_blast_results_df = all_blast_results_df[all_blast_results_df["Coverage"] > 0.95]

# choose best scaffold by pident
all_blast_results_df = all_blast_results_df.sort_values(by=["pident"], ascending=False)
all_blast_results_df = all_blast_results_df.drop_duplicates(subset=["qseqid"])

#rename columns
all_blast_results_df = all_blast_results_df.rename(columns={"qseqid":"ccf_scaffold_name", "sseqid":"assembled_scaffold_name"})

In [ ]:
public_qc_scaffolds_final_list = pd.read_csv(rootdir + "metadata/CCF_scaffold_QC_final_list.csv")
public_qc_scaffolds_final_list = public_qc_scaffolds_final_list.dropna()
public_qc_scaffolds_final_list = public_qc_scaffolds_final_list[public_qc_scaffolds_final_list["include"] == True]

In [ ]:
#merge with list of manually QC scaffolds
merged_public_assembled_scaffolds = public_qc_scaffolds_final_list.merge(all_blast_results_df, on="ccf_scaffold_name", how="inner")

In [ ]:
#save these results
merged_public_assembled_scaffolds.to_csv(rootdir + "metadata/merged_public_assembled_scaffolds.csv", index = False)

In [ ]:
#cating additional spacers from spacer expansion
print("cat {0}spacer_expansion/expansion_output/*/spacers.fasta >> {0}spacer_expansion/all_additional_spacers.fa".format(rootdir))

In [ ]:
#extracting spacers via pullseq for QC scaffolds
for x in merged_public_assembled_scaffolds["assembled_scaffold_name"]:
        pullseq = "pullseq -i {0}spacer_expansion/all_additional_spacers.fa -g {1} >> {0}spacer_expansion/qc_additional_spacers.fa".format(rootdir, x)
        os.system(pullseq)

In [ ]:
#combining lin xing + ccf spacers
print("cat {0}spacer_expansion/qc_additional_spacers.fa {0}/CCF_output_all_genomes/all_ccf_spacers_3_4_qc_scaffolds_only.fna > {0}All_qc_spacers.fna".format(rootdir))